In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
from collections import defaultdict 

import csv
from tqdm import tqdm
from pathlib import Path

In [11]:
path = Path.cwd()
file = pd.read_excel(path/'keyword.xlsx', engine = 'openpyxl')
category_list = file.values.tolist()

In [12]:
column = ['등록번호','참조번호','품명(사업명)','수요기관','사전규격공개일시','업체등록의견수']
df = pd.DataFrame(columns=column)
try:
    driver = webdriver.Chrome('./chromedriver')
    driver.implicitly_wait(2)

    for query in category_list:

        
        
        driver.get('https://www.g2b.go.kr:8081/ep/preparation/prestd/preStdSrch.do')
        task_dict = {'용역': 'taskClCds5'}
        for task in task_dict.values():
            checkbox = driver.find_element(By.ID, task)
            checkbox.click()
        # ID값이 prodNm인 태그 가져오기
        prodNm = driver.find_element(By.ID, 'prodNm')
        # 내용을 삭제
        prodNm.clear()
        # 검색어에 문자열 전달
        prodNm.send_keys(query)
        prodNm.send_keys(Keys.RETURN)
        
        # 목록수 100건 선택 (드롭다운)
        print(1)
        recordcountperpage = driver.find_element(By.NAME,'recordCountPerPage')
        selector = Select(recordcountperpage)
        selector.select_by_value('100')

        search_button = driver.find_element(By.CLASS_NAME, 'btn_mdl')
        search_button.click()
        print(2)
        # 검색 결과 확인
        elem = driver.find_element(By.CLASS_NAME,'results')
        div_list = elem.find_elements(By.TAG_NAME, 'tr')

        # 리스트 형태로 검색결과 저장
        
        print(3)
        results = []
        links = []
        for div in tqdm(div_list):
            results.append(div.text)
            row_data = [div.text.split('\n')]
            a_tags = div.find_elements(By.TAG_NAME,'a')
            if a_tags:
                a_tag = a_tags[0]
                link = a_tag.get_attribute('href')
                row_data.append([link])
            # create a new DataFrame with the data from this row
            
            df_row = pd.DataFrame(row_data)
            # append it to the overall df using pd.concat()
            df = pd.concat([df, df_row], ignore_index=True)
                #df = df.assign(링크 =links)

except Exception as e:
    print(e)
finally:  
    driver.quit()

C:\Users\JOONHY~1\AppData\Local\Temp/ipykernel_9640/1267538500.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


1
2
3


100%|██████████| 7/7 [00:02<00:00,  2.60it/s]


1
2
3


100%|██████████| 33/33 [00:04<00:00,  7.69it/s]


In [13]:
df

,등록번호,참조번호,품명(사업명),수요기관,사전규격공개일시,업체등록의견수,0,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,No. 등록번호 참조번호 품명(사업명) 수요기관 사전규격공개일시 업체등록의견수,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,1,1271610,구조연구본부-2569,해상특수교량 주탑부 드론 촬영 및 영상데이터 분석 용역,한국건설기술연구원,2023/03/24 17:05,0
2,NaN,NaN,NaN,NaN,NaN,NaN,javascript:toDtl('1271610');,None,None,None,None,None,None
3,NaN,NaN,NaN,NaN,NaN,NaN,2,1269766,122381048400001,2023년 북한정보 인공지능·빅데이터 분석모델 운영 및 유지관리 사업,통일부,2023/03/22 13:19,0
4,NaN,NaN,NaN,NaN,NaN,NaN,javascript:toDtl('1269766');,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,NaN,NaN,NaN,NaN,NaN,NaN,javascript:toDtl('1258187');,None,None,None,None,None,None
74,NaN,NaN,NaN,NaN,NaN,NaN,31,1257985,공고 경영지원실-2023-제52호,2023~2025 회계연도 회계감사 및 세무조정,국민건강보험공단,2023/02/28 17:53,0
75,NaN,NaN,NaN,NaN,NaN,NaN,javascript:toDtl('1257985');,None,None,None,None,None,None
76,NaN,NaN,NaN,NaN,NaN,NaN,32,1256763,한국언론진흥재단 제2023-66호,2022회계연도 한국언론진흥재단 법인회계 예산결산 검증 용역,한국언론진흥재단,2023/02/27 10:36,0


In [14]:
column = ['등록번호','참조번호','품명(사업명)','수요기관','사전규격공개일시','업체등록의견수']
new_df = df.iloc[:,7:14]
new_df.columns = column

In [15]:
new_df = new_df.dropna(subset=['등록번호'])
new_df['링크'] = 0
new_df = new_df.drop_duplicates(['등록번호'])
new_df = new_df.sort_values('사전규격공개일시',ascending = False)
new_df = new_df.reset_index(drop=True)
for i in range(len(new_df)):
    new_df['링크'][i] = 'https://www.g2b.go.kr:8082/ep/preparation/prestd/preStdDtl.do?preStdRegNo=' + new_df['등록번호'][i]


C:\Users\JOONHY~1\AppData\Local\Temp/ipykernel_9640/3527224224.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['링크'][i] = 'https://www.g2b.go.kr:8082/ep/preparation/prestd/preStdDtl.do?preStdRegNo=' + new_df['등록번호'][i]


In [16]:
from datetime import datetime 
today = datetime.today().strftime('%Y-%m-%d')

In [17]:
new_df.to_excel(f'사전규격공고_{today}.xlsx')

In [18]:
new_df

,등록번호,참조번호,품명(사업명),수요기관,사전규격공개일시,업체등록의견수,링크
0,1271610,구조연구본부-2569,해상특수교량 주탑부 드론 촬영 및 영상데이터 분석 용역,한국건설기술연구원,2023/03/24 17:05,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
1,1271237,S230324001,2023~2025 사업연도 회계감사용역,한국마사회,2023/03/24 10:54,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
2,1270972,공고 경영지원실-2023-제71호,"2022 회계연도 요양기관 의료비용 자료 수집, 계산 및 분석",국민건강보험공단,2023/03/23 18:05,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
3,1270718,23-42,2023년도 좋은 일자리 환경 구축을 위한 컨설팅(세무/회계) 수행 용역,사단법인제주경영자총협회,2023/03/23 15:38,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
4,1270025,경영지원실-1370,23~26년도 과학기술사업화진흥원 위탁정산 회계법인 모집 용역,과학기술일자리진흥원,2023/03/22 16:08,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
5,1270000,경영지원실-1371,과학기술사업화진흥원 회계·세무 자문용역,과학기술일자리진흥원,2023/03/22 15:36,1,https://www.g2b.go.kr:8082/ep/preparation/pres...
6,1269982,경영지원실-1372,‘23∼‘25년도 과학기술사업화진흥원 외부회계감사 용역,과학기술일자리진흥원,2023/03/22 15:29,2,https://www.g2b.go.kr:8082/ep/preparation/pres...
7,1269766,122381048400001,2023년 북한정보 인공지능·빅데이터 분석모델 운영 및 유지관리 사업,통일부,2023/03/22 13:19,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
8,1267293,관광기업창업팀-0317,14회 관광벤처기업 회계감사 및 사전 모니터링,한국관광공사,2023/03/17 10:07,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
9,1267034,아동청년과-3708,2023 자격증 취득과정-전산회계·세무 자격증반 운영 용역,서울특별시 서초구,2023/03/16 18:30,0,https://www.g2b.go.kr:8082/ep/preparation/pres...
